In [1]:
import pandas as pd
import os

In [2]:
def clear_columns(data, columns):
    for c in columns:
        mask1 = data[c] == '\\N'
        mask2 = data[c].isnull()
        data = data.drop(index=(data[mask1 | mask2]).index)
    return data

In [3]:
title_ratings = pd.read_table("db_files/title_ratings.tsv")
title_ratings = title_ratings[title_ratings['numVotes'] >= 500]
title_ratings.shape

(124190, 3)

In [4]:
title_basic = pd.read_table("db_files/title_basic.tsv")

C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_21060\2788150849.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basic = pd.read_table("db_files/title_basic.tsv")


In [5]:
df = title_basic.join(title_ratings.set_index('tconst'), on='tconst').dropna()
df.shape

(124190, 11)

In [6]:
df = df.drop(columns='titleType')
df = clear_columns(df, df.columns.drop(['endYear','runtimeMinutes']))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124142 entries, 0 to 9646786
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          124142 non-null  object 
 1   primaryTitle    124142 non-null  object 
 2   originalTitle   124142 non-null  object 
 3   isAdult         124142 non-null  object 
 4   startYear       124142 non-null  object 
 5   endYear         124142 non-null  object 
 6   runtimeMinutes  124142 non-null  object 
 7   genres          124142 non-null  object 
 8   averageRating   124142 non-null  float64
 9   numVotes        124142 non-null  float64
dtypes: float64(2), object(8)
memory usage: 10.4+ MB


In [7]:
df['startYear'] = df['startYear'].apply(int)
df = df[df.startYear >= 1980]

df['endYear'] = df.apply(
    lambda row:row['startYear'] if row['endYear'] == '\\N' else int(row['endYear']), axis=1
)

df['runtimeMinutes'] = df.apply(
    lambda row: 0 if row['runtimeMinutes'] == '\\N' else row['runtimeMinutes'], axis=1
)
df.shape

(109251, 10)

In [8]:
title_principals = pd.read_table("db_files/title_principals.tsv")
title_principals = title_principals.drop(columns=['job', 'characters'])
title_principals = title_principals.rename(columns={'nconst':'pnconst', 'category':'pcategory'})

In [9]:
df2 = df.join(title_principals.set_index('tconst'), on='tconst').dropna()

In [10]:
df2 = clear_columns(df2, title_principals.columns)
df2.head(50)

,tconst,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,ordering,pnconst,pcategory
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,10.0,nm0107463,editor
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,1.0,nm0000212,actress
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,2.0,nm0413168,actor
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,3.0,nm0000630,actor
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,4.0,nm0005227,actor
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,5.0,nm0003506,director
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,6.0,nm0737216,writer
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,7.0,nm0465298,producer
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,8.0,nm0448843,composer
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,9.0,nm0238698,cinematographer


In [11]:
df2.shape

(1061803, 13)

In [12]:
title_akas = pd.read_table("db_files/title_akas.tsv")

C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_21060\1489511692.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  title_akas = pd.read_table("db_files/title_akas.tsv")


In [13]:
title_akas.shape

(35065780, 8)

In [14]:
title_akas = title_akas[['titleId','ordering','region','types']]
title_akas = clear_columns(title_akas, title_akas.columns)
# title_akas = title_akas[['titleId','ordering','region','types','title','isOriginalTitle']]

In [15]:
m1 = title_akas['titleId'].isin(df2['tconst'])
m2 = title_akas['ordering'].isin(df2['ordering'])
title_akas = title_akas[m1 & m2]

In [16]:
title_akas = title_akas.rename(columns={'titleId':'tconst'})
# title_akas = title_akas.rename(columns={'titleId':'tconst', 'title':'akas_title'})
title_akas.shape

(456668, 4)

In [17]:
df3 = df2.join(title_akas.set_index(['tconst','ordering']), on=['tconst','ordering']).dropna()
df3.shape

(448461, 15)

In [18]:
df3.head(50)

,tconst,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,ordering,pnconst,pcategory,region,types
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,10.0,nm0107463,editor,IT,imdbDisplay
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,2.0,nm0413168,actor,EG,imdbDisplay
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,3.0,nm0000630,actor,RS,imdbDisplay
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,4.0,nm0005227,actor,MX,imdbDisplay
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,5.0,nm0003506,director,SK,imdbDisplay
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,6.0,nm0737216,writer,CZ,imdbDisplay
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,7.0,nm0465298,producer,RU,imdbDisplay
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,8.0,nm0448843,composer,UA,imdbDisplay
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,9.0,nm0238698,cinematographer,AU,imdbDisplay
38011,tt0038687,Let There Be Light,Let There Be Light,0,1980,1980,58,"Documentary,War",7.4,1916.0,1.0,nm0404158,actor,FR,imdbDisplay


In [19]:
name_basics = pd.read_csv('db_files/name_basics.tsv', delimiter='\t')
name_basics = clear_columns(name_basics, name_basics.columns.drop(['deathYear']))
name_basics['deathYear'] = name_basics['deathYear'].replace({'\\N':'2024'})

In [20]:
name_basics['knownForTitles'] = name_basics['knownForTitles'].apply(lambda x : x.split(',')).explode('knownForTitles')
name_basics = name_basics[name_basics['knownForTitles'].isin(df3.tconst)]

In [21]:
name_basics = name_basics.rename(columns={'knownForTitles':'tconst'})
name_basics.shape

(73958, 6)

In [22]:
df4 = df3.join(name_basics.set_index('tconst'), on='tconst')
df4.shape

(832170, 20)

In [23]:
df4.head(50)

,tconst,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,ordering,pnconst,pcategory,region,types,nconst,primaryName,birthYear,deathYear,primaryProfession
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,10.0,nm0107463,editor,IT,imdbDisplay,nm0179321,Noreen Corcoran,1943,2016,"actress,soundtrack"
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,10.0,nm0107463,editor,IT,imdbDisplay,nm0228072,Françoise Diot,1914,2013,"editor,editorial_department,sound_department"
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,10.0,nm0107463,editor,IT,imdbDisplay,nm0755977,Maksim Sagalovich,1915,1997,writer
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,10.0,nm0107463,editor,IT,imdbDisplay,nm0790590,Haleigh Sheehan,1983,2024,actress
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,10.0,nm0107463,editor,IT,imdbDisplay,nm0866390,Peter Tomarken,1942,2006,"actor,producer,writer"
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,2.0,nm0413168,actor,EG,imdbDisplay,nm0179321,Noreen Corcoran,1943,2016,"actress,soundtrack"
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,2.0,nm0413168,actor,EG,imdbDisplay,nm0228072,Françoise Diot,1914,2013,"editor,editorial_department,sound_department"
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,2.0,nm0413168,actor,EG,imdbDisplay,nm0755977,Maksim Sagalovich,1915,1997,writer
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,2.0,nm0413168,actor,EG,imdbDisplay,nm0790590,Haleigh Sheehan,1983,2024,actress
34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,86095.0,2.0,nm0413168,actor,EG,imdbDisplay,nm0866390,Peter Tomarken,1942,2006,"actor,producer,writer"


In [24]:
df4 = clear_columns(df4, df4.columns)

In [25]:
df4 = df4.astype({
    'isAdult':'int32',
    'ordering':'int32',
    'birthYear':'int32',
    'deathYear':'int32',
})

In [26]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558172 entries, 34803 to 9646786
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   tconst             558172 non-null  object 
 1   primaryTitle       558172 non-null  object 
 2   originalTitle      558172 non-null  object 
 3   isAdult            558172 non-null  int32  
 4   startYear          558172 non-null  int64  
 5   endYear            558172 non-null  int64  
 6   runtimeMinutes     558172 non-null  object 
 7   genres             558172 non-null  object 
 8   averageRating      558172 non-null  float64
 9   numVotes           558172 non-null  float64
 10  ordering           558172 non-null  int32  
 11  pnconst            558172 non-null  object 
 12  pcategory          558172 non-null  object 
 13  region             558172 non-null  object 
 14  types              558172 non-null  object 
 15  nconst             558172 non-null  object 
 1

In [27]:
df = df4  # last updated df
os.makedirs('db_files', exist_ok=True)
df.to_csv('db_files/db_cleaned.csv')